In [248]:
from keystonexl.client import PredatorClient
from keystonexl.base import PitchSide
import numpy as np
import enum
from math import isnan
%load_ext autotime

The autotime extension is already loaded. To reload it, use:
  %reload_ext autotime
time: 2.8 ms


In [2]:
base_path = '/Users/jason.katz/Downloads'
predator_client = PredatorClient.from_paths(moment_path='{}/SAMPLE.hudlproj/momentsSave.json'.format(base_path), 
                                            tracking_path='{}/SAMPLE.hudlproj/trackingSave.json'.format(base_path))

time: 4.22 s


In [68]:
player_ids = [column[11:] for column in predator_client.moment_tag_df.columns if 'player_id__p_' in column]
all_goalkeepers = [player for player in predator_client.players if player['player_position'] == 'Goalkeeper']
goalkeepers = [goalkeeper for idx, goalkeeper in enumerate(all_goalkeepers) if goalkeeper['hudl_athlete_id'] in player_ids]
for goalkeeper in goalkeepers:
    goalkeeper_id = goalkeeper['hudl_athlete_id']
    if goalkeeper['player_team']['team_type'] == 'home':
        goalkeeper_home_idx = predator_client.lookup_from_df('queryable_id', goalkeeper_id, 'tracking_index')
    else:
        goalkeeper_away_idx = predator_client.lookup_from_df('queryable_id', goalkeeper_id, 'tracking_index')

time: 9.64 ms


In [3]:
class AlwaysTrue(enum.Enum):
    yes = 1
    no = 0

class FreeKick(enum.Enum):
    yes = 1
    no = 0
    
class Corner(enum.Enum):
    yes = 1
    no = 0
    
class Cross(enum.Enum):
    yes = 1
    no = 0
    
class ThrowIn(enum.Enum):
    yes = 1
    no = 0
    
class Header(enum.Enum):
    yes = 1
    no = 0
    
class Made(enum.Enum):
    yes = 1
    no = 0

time: 10.4 ms


In [253]:
test = Game(predator_client)

time: 1.26 s


In [252]:
class Game(object):
    def __init__(self, predator_client):
        self.predator_client = predator_client
        self.game_id = np.random.rand()
        self.shots = self.extract_data()
        self.first_shot = self.shots[0]   
        self.first_shot_period_num = self.first_shot['period_num']
        self.first_shot_team_id = predator_client.get_value(self.first_shot, 'team_id')
        self.first_shot_team_type = predator_client.team_type(self.first_shot_team_id)
        self.first_shot_goal_side = predator_client.estimate_defending_half(self.first_shot_team_type, 
                                                                            self.first_shot_period_num)
        self.goalkeeper_home_idx, self.goalkeeper_away_idx = self.get_goalkeepers()
        self.player_idxs = self.get_all_player_idxs()
        self.shots = [Shot(shot, self.game_id, self.predator_client, self.first_shot_period_num, self.first_shot_team_type, 
                           self.first_shot_goal_side, self.goalkeeper_home_idx, self.goalkeeper_away_idx, self.player_idxs) for shot in self.shots]
        
    def extract_data(self):
        extracted_moments = self.predator_client.moments
        shots = []
        for idx, moment in enumerate(extracted_moments):
            extracted_moments[idx]['index'] = idx
            frame = self.predator_client.frame_from_time(moment['period_num'], moment['end_ms'])
            try:
                player_id = self.predator_client.get_value(moment, 'player_id')
                player_idx = self.predator_client.lookup_from_df('queryable_id', player_id, 'tracking_index')
                extracted_moments[idx]['x'] = self.predator_client.tracking_array[frame, player_idx, :2][0]
            except:
                if moment['end_ms'] < 1.0:
                    extracted_moments[idx]['x'] = 0.0
                else:
                    extracted_moments[idx]['x'] = extracted_moments[idx-1]['x']
            if self.predator_client.has_tag(moment, 'shot'):
                moment['speed_3'] = abs(extracted_moments[idx-3]['x'] - moment['x']) / (moment['end_ms'] - extracted_moments[idx-3]['end_ms']) * 1000
                moment['speed_5'] = abs(extracted_moments[idx-5]['x'] - moment['x']) / (moment['end_ms'] - extracted_moments[idx-5]['end_ms']) * 1000
                shots.append(moment)
        return shots
    
    def get_goalkeepers(self):
        player_ids = [column[11:] for column in self.predator_client.moment_tag_df.columns if 'player_id__p_' in column]
        
        players = [self.predator_client.lookup_from_df('queryable_id', player, 'tracking_index') for player in player_ids]
        all_goalkeepers = [player for player in self.predator_client.players if player['player_position'] == 'Goalkeeper']
        goalkeepers = [goalkeeper for idx, goalkeeper in enumerate(all_goalkeepers) if goalkeeper['hudl_athlete_id'] in player_ids]
        for goalkeeper in goalkeepers:
            goalkeeper_id = goalkeeper['hudl_athlete_id']
            if goalkeeper['player_team']['team_type'] == 'home':
                goalkeeper_home_idx = self.predator_client.lookup_from_df('queryable_id', goalkeeper_id, 'tracking_index')
            else:
                goalkeeper_away_idx = self.predator_client.lookup_from_df('queryable_id', goalkeeper_id, 'tracking_index')
        return goalkeeper_home_idx, goalkeeper_away_idx
    
    def get_all_player_idxs(self):
        player_idxs = {'home': [], 'away': []}
        for player in self.predator_client.players:
            player_idx = self.predator_client.lookup_from_df('queryable_id', player['hudl_athlete_id'], 'tracking_index')
            if player_idx != -1:
                team = player['player_team']['team_type']
                player_idxs[team].append(player_idx)
        return player_idxs

time: 61.3 ms


In [259]:
class Shot(object):
    def __init__(self, shot, game_id, predator_client, first_shot_period_num, first_shot_team_type, first_shot_goal_side,
                goalkeeper_home_idx, goalkeeper_away_idx, player_idxs):
        self.shot = shot
        self.game_id = game_id
        self.predator_client = predator_client
        self.first_shot_period_num = first_shot_period_num
        self.first_shot_team_type = first_shot_team_type
        self.first_shot_goal_side = first_shot_goal_side
        self.goalkeeper_home_idx = goalkeeper_home_idx
        self.goalkeeper_away_idx = goalkeeper_away_idx
        self.player_idxs = player_idxs
    
            
    @property
    def always_true(self):
        return AlwaysTrue.yes
    
    @property
    def teams(self):
        return predator_client.teams
    
    def get_team_idx(self, team_type):
        for idx, team in enumerate(self.teams):
            if team['team_type'] == team_type:
                return idx
    
    @property
    def home_team_idx(self):
        return self.get_team_idx('home')
    
    @property
    def away_team_idx(self):
        return self.get_team_idx('away')
    
    @property
    def frame(self):
        return predator_client.frame_from_time(self.shot['period_num'], self.shot['end_ms'])
    
    @property
    def period_num(self):
        return self.shot['period_num']
    
    @property
    def team_id(self):
        return predator_client.get_value(self.shot, 'team_id')
    
    @property
    def home_team_id(self):
        return self.teams[self.home_team_idx]['hudl_team_id']
    
    @property
    def home_team_name(self):
        return self.teams[self.home_team_idx]['team_name']
    
    @property
    def away_team_id(self):
        return self.teams[self.away_team_idx]['hudl_team_id']
    
    @property
    def away_team_name(self):
        return self.teams[self.away_team_idx]['team_name']
        
    @property
    def team_type(self):
        return predator_client.team_type(self.team_id)
    
    @property
    def opposing_team_type(self):
        if self.team_type == 'home':
            return 'away'
        else:
            return 'home'
    
    @property
    def goal_side(self):
        if self.period_num == self.first_shot_period_num: 
            if self.team_type == self.first_shot_team_type:
                return self.first_shot_goal_side
            else:
                if self.first_shot_goal_side.name == 'left':
                    return PitchSide.right
                else:
                    return PitchSide.left
        elif self.period_num == 1 or self.period_num == 2:
            if self.team_type == self.first_shot_team_type:
                if self.first_shot_goal_side.name == 'left':
                    return PitchSide.right
                else:
                    return PitchSide.left
            else:
                return self.first_shot_goal_side
        else:
            return predator_client.estimate_defending_half(self.team_type, self.period_num )
        
    @property
    def goal_x(self):
        if self.goal_side == PitchSide.right:
            return predator_client.pitch_width * -.5
        else:
            return predator_client.pitch_width * .5
        
    @property
    def goal_coords(self):
        return np.array([self.goal_x, 0])
    
    @property
    def player_id(self):
        return predator_client.get_value(self.shot, 'player_id')
        
    @property
    def player_idx(self):
        return predator_client.lookup_from_df('queryable_id', self.player_id, 'tracking_index')
    
    @property
    def player_coords(self):
        return predator_client.tracking_array[self.frame, self.player_idx, :2]
    
    @property
    def goalkeeper_coords(self):
        if self.team_type == 'home':
            return predator_client.tracking_array[self.frame, self.goalkeeper_away_idx, :2]
        else:
            return predator_client.tracking_array[self.frame, self.goalkeeper_home_idx, :2]
        
    @property
    def opposing_player_coords(self):
        return [self.predator_client.tracking_array[self.frame, player, :2] for player in self.player_idxs[self.opposing_team_type] if not isnan(self.predator_client.tracking_array[self.frame, player, 1])]
    
    @property
    def teamates_coords(self):
        return [self.predator_client.tracking_array[self.frame, player, :2] for player in self.player_idxs[self.team_type] if not isnan(self.predator_client.tracking_array[self.frame, player, 1])]
    
    @property
    def goalkeeper_x(self):
        return self.goalkeeper_coords[0]
    
    @property
    def goalkeeper_y(self):
        return self.goalkeeper_coords[1]
    
    @property
    def x(self):
        return self.player_coords[0]
    
    @property
    def y(self):
        return self.player_coords[1]
        
    @property
    def distance(self):
        return np.linalg.norm(self.player_coords - self.goal_coords)
    
    def has_attribute(self, event_type):
        in_list = False
        while not in_list:
            for descriptor in self.shot['tags']:
                if descriptor['key'] == event_type:
                    in_list = True
            break
        return in_list
    
    @property
    def free_kick(self):
        if self.has_attribute('free kick'):
            return FreeKick.yes
        else:
            return FreeKick.no
        
    @property
    def corner(self):
        if self.has_attribute('corner'):
            return Corner.yes
        else:
            return Corner.no
        
    @property
    def cross(self):
        if self.has_attribute('cross'):
            return Cross.yes
        else:
            return Cross.no
        
    @property
    def throw_in(self):
        if self.has_attribute('throw in'):
            return ThrowIn.yes
        else:
            return ThrowIn.no
        
    @property
    def header(self):
        if self.has_attribute('header'):
            return Header.yes
        else:
            return Header.no
        
    @property
    def made(self):
        if self.has_attribute('goal'):
            return Made.yes
        else:
            return Made.no
        
    @property
    def speed_3(self):
        return self.shot['speed_3']
    
    @property
    def speed_5(self):
        return self.shot['speed_5']

time: 279 ms
